## k-近邻算法
###  kNN属于较为典型的监督学习算法，即已知训练集中数据的标签。
> 假设存在训练集***R***，将输入的数据与训练集中的样本作特征值的比对，计算他们之间的距离(欧式距离、曼哈顿距离等)，
> 选取距离最小(也就是相似度最高)的k个样本，将这k个样本的标签取出，其中出现频率最高的标签即为输入数据的预测标签。

In [ ]:
from numpy import *
import operator
def kNearestNeighborClassifier(inputData, dataSet, labels, k):
    """
    k-近邻算法
    :param inputData: 输入的数据
    :param dataSet:   训练集
    :param labels:    标签列表
    :param k:         近邻数量
    :return:          分类结果
    """
    # 获取数据集的大小，样本集为样本数据的特征矩阵，该特征矩阵的每一个行向量都代表了一个样本，故此处只需要获得行数即可表示数据集的大小
    dataSize = dataSet.shape[0]
    
    # 计算距离，这里采用欧式距离
    diffMat = tile(inputData, (dataSize, 1)) - dataSet
    squaredDiffMat = diffMat ** 2
    squaredDistances = squaredDiffMat.sum(axis=1)
    distances = squaredDistances ** 0.5
    
    # 距离从小到大排序
    sortedDistances = distances.argsort()
    classCount = {}
    # 统计k个近邻中的类别
    for i in range(k):
        voteLabel = labels[sortedDistances[i]]
        classCount[voteLabel] = classCount.get(voteLabel, 0)
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

## 归一化思想
> 从算法原理可以看出，kNN算法在计算距离时，(特别是欧氏距离)如果特征值向量中某一维度的特征值比其他维度的要大得多时，
> 这个特征值对计算出的距离(相似度)的影响会更大，导致其他特征值的比重降低，因此需要对特征值进行归一化处理。   
> 公式为: ***val_norm = (val - min_val)/(max_val - min_val)***

In [ ]:
def normalize(dataSet):
    """
    归一化处理
    :param dataSet: 原始的数据集
    :return: 归一化后的数据集，范围向量，最小值向量，后两个参数用于输入数据的归一化
    """
    # 计算每一列(即每个特征值列向量的最大值和最小值)
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    # 求出每列特征值的取值范围
    ranges = maxVals - minVals
    # 归一化处理，创建一个空的归一化矩阵
    normDataSet = zeros(shape(dataSet))
    # 获得样本数量
    m = dataSet.shape[0]
    # 所有的特征值都减去当前列特征值最小值，然后除于当前特征值范围，得到归一化值
    normDataSet = dataSet - tile(minVals, (m, 1))
    normDataSet = normDataSet / tile(ranges, (m, 1))
    return normDataSet, ranges, minVals